# ATAC-Seq analysis example
### Original authors
Délara Sabéran-Djoneidi, Anne-Laure Schang, Kate Wooley-Allen, Sascha Ott
### Update
Alix Silvert, Magali Hennion
## Intro and general description

This document aims to be a detailed example of an ATAC-Seq analysis used in a research context. The goal here is to help you draft your own experiments from this particular base, by walking you through the steps we took and by pointing out the parts where the specificity of your experimental set will require some tinkering.

## Clone the git repository

The documents related to this tutorial are available at https://github.com/parisepigenetics/ATAC-seq. The rest of this description assumes that you have cloned this repository (using `git clone`) and that you are in `ATAC-seq` folder. The `pwd` command allows you to check where you are. 

In [3]:
pwd

/home/mag/ATACseq/TestJupyterNotebook
(ATAC-Seq) 

: 1

In [4]:
## if you're not in the ATAC-seq folder, use cd to enter it.
## cd PATHto/ATAC-seq
cd ATAC-seq

(ATAC-Seq) 

: 1

## Load Conda environment

In order to be sure to have the appropriate tools to run this analysis, we provide a Conda environment summarizing them. In order to use it, please download [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/download.html) and run the following line.


In [ ]:
conda env create -f conda_environment.yml

All the remaining of the analysis must be run in the ATAC-seq Conda environment. You have to activate the environment. 

In [1]:
conda activate ATAC-Seq

(ATAC-Seq) 

: 1

## Download test dataset

To test this tutorial, you can download a small test dataset. 

In [4]:
wget https://raw.githubusercontent.com/nf-core/test-datasets/atacseq/testdata/SRR182215{3,4,7,8}_1.fastq.gz https://raw.githubusercontent.com/nf-core/test-datasets/atacseq/testdata/SRR182215{3,4,7,8}_2.fastq.gz -P data/raw_reads/

--2021-09-24 15:42:10--  https://raw.githubusercontent.com/nf-core/test-datasets/atacseq/testdata/SRR1822153_1.fastq.gz
SSL_INIT
Certificat de l’autorité de certification « /etc/ssl/certs/ca-certificates.crt » chargé
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 5272547 (5,0M) [application/octet-stream]
Sauvegarde en : « data/raw_reads/SRR1822153_1.fastq.gz »

SRR1822153_1.fastq. 100%[===================>]   5,03M  3,46MB/s    ds 1,5s    

2021-09-24 15:42:11 (3,46 MB/s) — « data/raw_reads/SRR1822153_1.fastq.gz » sauvegardé [5272547/5272547]

--2021-09-24 15:42:11--  https://raw.githubusercontent.com/nf-core/test-datasets/atacseq/testdata/SRR1822154_1.fastq.gz
SSL_INIT
Réutilisation de la connexion existante à raw.githubusercontent.com:443.
requête

: 1

And the corresponding reference files. 

In [5]:
ls data/raw_reads

SRR1822153_1.fastq.gz  SRR1822154_2.fastq.gz  SRR1822158_1.fastq.gz
SRR1822153_2.fastq.gz  SRR1822157_1.fastq.gz  SRR1822158_2.fastq.gz
SRR1822154_1.fastq.gz  SRR1822157_2.fastq.gz
(ATAC-Seq) 

: 1

In [6]:
wget https://github.com/nf-core/test-datasets/raw/atacseq/reference/genome.fa -P reference

--2021-09-24 15:42:42--  https://github.com/nf-core/test-datasets/raw/atacseq/reference/genome.fa
SSL_INIT
Certificat de l’autorité de certification « /etc/ssl/certs/ca-certificates.crt » chargé
Résolution de github.com (github.com)… 140.82.121.3
Connexion à github.com (github.com)|140.82.121.3|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://raw.githubusercontent.com/nf-core/test-datasets/atacseq/reference/genome.fa [suivant]
--2021-09-24 15:42:43--  https://raw.githubusercontent.com/nf-core/test-datasets/atacseq/reference/genome.fa
SSL_INIT
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 12359807 (12M) [text/plain]
Sauvegarde en : « reference/genome.fa.1 »

genome.fa.1         100%[==================

: 1

## Initial quality control

In order to see how much the pre-processing improves the data, it is a good practice to look at various statistics on the raw dataset. FastQC provides a good summary of those.

In [7]:
mkdir -p output/stats/raw_fastQC/
fastqc -o output/stats/raw_fastQC/ data/raw_reads/SRR182215{3,4,7,8}_1.fastq.gz data/raw_reads/SRR182215{3,4,7,8}_2.fastq.gz 

Started analysis of SRR1822153_1.fastq.gz
Approx 5% complete for SRR1822153_1.fastq.gz
Approx 10% complete for SRR1822153_1.fastq.gz
Approx 15% complete for SRR1822153_1.fastq.gz
Approx 20% complete for SRR1822153_1.fastq.gz
Approx 25% complete for SRR1822153_1.fastq.gz
Approx 30% complete for SRR1822153_1.fastq.gz
Approx 35% complete for SRR1822153_1.fastq.gz
Approx 40% complete for SRR1822153_1.fastq.gz
Approx 45% complete for SRR1822153_1.fastq.gz
Approx 50% complete for SRR1822153_1.fastq.gz
Approx 55% complete for SRR1822153_1.fastq.gz
Approx 60% complete for SRR1822153_1.fastq.gz
Approx 65% complete for SRR1822153_1.fastq.gz
Approx 70% complete for SRR1822153_1.fastq.gz
Approx 75% complete for SRR1822153_1.fastq.gz
Approx 80% complete for SRR1822153_1.fastq.gz
Approx 85% complete for SRR1822153_1.fastq.gz
Approx 90% complete for SRR1822153_1.fastq.gz
Approx 95% complete for SRR1822153_1.fastq.gz
Approx 100% complete for SRR1822153_1.fastq.gz
Analysis complete for SRR1822153_1.fas

: 1

If you have multiple samples, multiQC is a good way to look at the global quality over your samples.

In [8]:
multiqc output/stats/raw_fastQC -o output/stats/raw_fastQC


  /// ]8;id=751489;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.11

|           multiqc | Search path : /home/mag/ATACseq/TestJupyterNotebook/ATAC-seq/output/stats/raw_fastQC
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 16/16   
|            fastqc | Found 8 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : output/stats/raw_fastQC/multiqc_report.html
|           multiqc | Data        : output/stats/raw_fastQC/multiqc_data
|           multiqc | MultiQC complete
(ATAC-Seq) 

: 1

## Trimming

The fastq files are trimmed using Trimmomatic v0.39 to remove any adapter sequences in the reads caused by read through associated with DNA fragments shorter in size than the read length being sequenced.

This particular step need to be adapted depending on your exact data and need. We found these options to be the most efficient in the case of ATAC-Seq. If you want to know more we encourage you to read the [documentation](http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/TrimmomaticManual_V0.32.pdf) of the Trimmomatic tool in order to find the best possible use for your case.


In [11]:
mkdir -p output/trimmed_files
for i in {3,4,7,8}; 
do trimmomatic PE -threads 2 data/raw_reads/SRR182215${i}_1.fastq.gz data/raw_reads/SRR182215${i}_2.fastq.gz output/trimmed_files/SRR182215${i}_1_trimmed.fastq output/trimmed_files/SRR182215${i}_1_trimmed_unpaired.fa output/trimmed_files/SRR182215${i}_2_trimmed.fastq output/trimmed_files/SRR182215${i}_2_trimmed_unpaired.fa ILLUMINACLIP:data/adapters/NexteraPE-PE.fa:2:30:10:1:true TRAILING:3 SLIDINGWINDOW:4:15;
done

TrimmomaticPE: Started with arguments:2004l
 -threads 2 data/raw_reads/SRR1822153_1.fastq.gz data/raw_reads/SRR1822153_2.fastq.gz output/trimmed_files/SRR1822153_1_trimmed.fastq output/trimmed_files/SRR1822153_1_trimmed_unpaired.fa output/trimmed_files/SRR1822153_2_trimmed.fastq output/trimmed_files/SRR1822153_2_trimmed_unpaired.fa ILLUMINACLIP:data/adapters/NexteraPE-PE.fa:2:30:10:1:true TRAILING:3 SLIDINGWINDOW:4:15
Using PrefixPair: 'AGATGTGTATAAGAGACAG' and 'AGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTCCGAGCCCACGAGAC'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTGACGCTGCCGACGA'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 100000 Both Surviving: 99529 (99,53%) Forward Only Surviving: 419 (0,42%) R

: 1

## Alignement

This step must also be adapted to your particular experiment, namely the genomes used may vary and the aligner used as well as the alignment options can be optimized.

In our case, alignment is performed by bowtie2.4.4. 

### Bowtie2 Index

In order to align reads to genome(s) and particular cellular compartments (i.e. mitochondria), it is necessary to build a bowtie index for each of them and potential confounders. In the end, only the reads mapping solely to the target of interest will be saved. In our case, the mitochondrial hits will be removed.

Bowtie2 necessitate and index to be built before the alignment can be done. By default, the exact method used is chosen depending on the machine doing the computation, please read the [documentation of bowtie2-build](http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml#the-bowtie2-build-indexer) for further information, and be careful when repeating a study on different machines.

If you already have a bowtie index built for this type of alignment, this step can be skipped.

In this example, we start from the fasta of a complete genome that we split into chromosomes using a small bash script.

In [ ]:
bash scripts/split_reference.sh reference/genome.fa

Then we build one index for the mitochondrie, and one index for the other chromosomes. 

In [ ]:
cd reference
bowtie2-build genome.fa_MT yeast_MT > MT_index.log
bowtie2-build genome.fa_I,genome.fa_II,genome.fa_III,genome.fa_IV,genome.fa_V,genome.fa_VI,genome.fa_VII,genome.fa_VIII,genome.fa_IX,genome.fa_X,genome.fa_XI,genome.fa_XII,genome.fa_XIII,genome.fa_XIV,genome.fa_XV,genome.fa_XVI yeast_NC > NC_index.log
cd .. # to go back to ATAC-seq folder. 

### Alignment

#### Removal of Mitochondrial DNA

Let's breakdown the various options used:

* `-p 4` We are using 4 threads for alignment. This option can also be accessed via `--threads`.
* `-X 2000` The maximum fragment length accepted between paired end. This number should be adapted depending on your sequencing protocol.
* `--very-sensitive` Preset option corresponding to `-D 20 -R 3 -N 0 -L 20 -i S,1,0.50`.
* `-x` Selected bowtie index.
*  `-1 and -2` The input fastq files in the case of paired-end alignment
*  `-S` File in which to write the output SAM file of the aligned reads. This file is not useful but printing it allows us to use less memory.
*  `--un-conc` We only keep pairs that DIDN'T align on mitochondrial genome.

for more informations and option, you can run `bowtie2 --help`.

In [33]:
mkdir -p tmp
for i in {3,4,7,8}; 
do bowtie2 -p 4 -X 2000 --very-sensitive -x reference/yeast_MT -1 output/trimmed_files/SRR182215${i}_1_trimmed.fastq -2 output/trimmed_files/SRR182215${i}_2_trimmed.fastq -S tmp/SRR182215${i}_MT.sam --un-conc-gz tmp/SRR182215${i}_notMT.fastq.gz;
done

99529 reads; of these:
  99529 (100.00%) were paired; of these:
    98451 (98.92%) aligned concordantly 0 times
    851 (0.86%) aligned concordantly exactly 1 time
    227 (0.23%) aligned concordantly >1 times
    ----
    98451 pairs aligned concordantly 0 times; of these:
      0 (0.00%) aligned discordantly 1 time
    ----
    98451 pairs aligned 0 times concordantly or discordantly; of these:
      196902 mates make up the pairs; of these:
        195868 (99.47%) aligned 0 times
        78 (0.04%) aligned exactly 1 time
        956 (0.49%) aligned >1 times
1.60% overall alignment rate


99538 reads; of these:
  99538 (100.00%) were paired; of these:
    98497 (98.95%) aligned concordantly 0 times
    855 (0.86%) aligned concordantly exactly 1 time
    186 (0.19%) aligned concordantly >1 times
    ----
    98497 pairs aligned concordantly 0 times; of these:
      1 (0.00%) aligned discordantly 1 time
    ----
    98496 pairs aligned 0 times concordantly or discordantly; of these:
      196992 mates make up the pairs; of these:
        195953 (99.47%) aligned 0 times
        83 (0.04%) aligned exactly 1 time
        956 (0.49%) aligned >1 times
1.57% overall alignment rate


99576 reads; of these:
  99576 (100.00%) were paired; of these:
    99252 (99.67%) aligned concordantly 0 times
    219 (0.22%) aligned concordantly exactly 1 time
    105 (0.11%) aligned concordantly >1 times
    ----
    99252 pairs aligned concordantly 0 times; of these:
      1 (0.00%) aligned discordantly 1 time
    ----
    99251 pairs aligned 0 times concordantly or discordantly; of these:
      198502 mates make up the pairs; of these:
        197627 (99.56%) aligned 0 times
        65 (0.03%) aligned exactly 1 time
        810 (0.41%) aligned >1 times
0.77% overall alignment rate


99484 reads; of these:
  99484 (100.00%) were paired; of these:
    99122 (99.64%) aligned concordantly 0 times
    248 (0.25%) aligned concordantly exactly 1 time
    114 (0.11%) aligned concordantly >1 times
    ----
    99122 pairs aligned concordantly 0 times; of these:
      0 (0.00%) aligned discordantly 1 time
    ----
    99122 pairs aligned 0 times concordantly or discordantly; of these:
      198244 mates make up the pairs; of these:
        197057 (99.40%) aligned 0 times
        78 (0.04%) aligned exactly 1 time
        1109 (0.56%) aligned >1 times
0.96% overall alignment rate
(ATAC-Seq) 

: 1

This file containing the alignment on the mitochondria won't be used, so we remove it. 

In [13]:
rm tmp/SRR182215{3,4,7,8}_MT.sam

(ATAC-Seq) 

: 1

#### Alignment on chromosomal genome

We now align the remaining reads on the nuclear genome, in a very similar manner. 

In [34]:
ls tmp

SRR1822153_MT.sam            SRR1822157_MT.sam
SRR1822153_notMT_1.fastq.gz  SRR1822157_notMT_1.fastq.gz
SRR1822153_notMT_2.fastq.gz  SRR1822157_notMT_2.fastq.gz
SRR1822153_notMT.fastq.1.gz  SRR1822157_notMT.fastq.1.gz
SRR1822153_notMT.fastq.2.gz  SRR1822157_notMT.fastq.2.gz
SRR1822154_MT.sam            SRR1822158_MT.sam
SRR1822154_notMT_1.fastq.gz  SRR1822158_notMT_1.fastq.gz
SRR1822154_notMT_2.fastq.gz  SRR1822158_notMT_2.fastq.gz
SRR1822154_notMT.fastq.1.gz  SRR1822158_notMT.fastq.1.gz
SRR1822154_notMT.fastq.2.gz  SRR1822158_notMT.fastq.2.gz
(ATAC-Seq) 

: 1

In [35]:
mkdir output/aligned_files
for i in {3,4,7,8}; 
do echo ${i};
   bowtie2 -p 4 -X 2000 --very-sensitive -x reference/yeast_NC -1 tmp/SRR182215${i}_notMT.fastq.1.gz -2 tmp/SRR182215${i}_notMT.fastq.2.gz -S output/aligned_files/SRR182215${i}_notMT_aligned.sam;
done

mkdir: impossible de créer le répertoire « output/aligned_files »: Le fichier existe
3[?2004h(ATAC-Seq) 


98451 reads; of these:
  98451 (100.00%) were paired; of these:
    4877 (4.95%) aligned concordantly 0 times
    48367 (49.13%) aligned concordantly exactly 1 time
    45207 (45.92%) aligned concordantly >1 times
    ----
    4877 pairs aligned concordantly 0 times; of these:
      21 (0.43%) aligned discordantly 1 time
    ----
    4856 pairs aligned 0 times concordantly or discordantly; of these:
      9712 mates make up the pairs; of these:
        8711 (89.69%) aligned 0 times
        210 (2.16%) aligned exactly 1 time
        791 (8.14%) aligned >1 times
95.58% overall alignment rate
4


98497 reads; of these:
  98497 (100.00%) were paired; of these:
    7400 (7.51%) aligned concordantly 0 times
    39882 (40.49%) aligned concordantly exactly 1 time
    51215 (52.00%) aligned concordantly >1 times
    ----
    7400 pairs aligned concordantly 0 times; of these:
      22 (0.30%) aligned discordantly 1 time
    ----
    7378 pairs aligned 0 times concordantly or discordantly; of these:
      14756 mates make up the pairs; of these:
        13630 (92.37%) aligned 0 times
        211 (1.43%) aligned exactly 1 time
        915 (6.20%) aligned >1 times
93.08% overall alignment rate
7


99252 reads; of these:
  99252 (100.00%) were paired; of these:
    5615 (5.66%) aligned concordantly 0 times
    59187 (59.63%) aligned concordantly exactly 1 time
    34450 (34.71%) aligned concordantly >1 times
    ----
    5615 pairs aligned concordantly 0 times; of these:
      17 (0.30%) aligned discordantly 1 time
    ----
    5598 pairs aligned 0 times concordantly or discordantly; of these:
      11196 mates make up the pairs; of these:
        10422 (93.09%) aligned 0 times
        216 (1.93%) aligned exactly 1 time
        558 (4.98%) aligned >1 times
94.75% overall alignment rate
8


99122 reads; of these:
  99122 (100.00%) were paired; of these:
    5780 (5.83%) aligned concordantly 0 times
    60336 (60.87%) aligned concordantly exactly 1 time
    33006 (33.30%) aligned concordantly >1 times
    ----
    5780 pairs aligned concordantly 0 times; of these:
      25 (0.43%) aligned discordantly 1 time
    ----
    5755 pairs aligned 0 times concordantly or discordantly; of these:
      11510 mates make up the pairs; of these:
        10564 (91.78%) aligned 0 times
        249 (2.16%) aligned exactly 1 time
        697 (6.06%) aligned >1 times
94.67% overall alignment rate
(ATAC-Seq) 

: 1

We remove temporary files. 

In [38]:
rm -fr tmp/

(ATAC-Seq) 

: 1

We now sort the sam file, convert it to bam, and then index it.

In [40]:
ls output/aligned_files

SRR1822153_notMT_aligned.sam  SRR1822157_notMT_aligned.sam
SRR1822154_notMT_aligned.sam  SRR1822158_notMT_aligned.sam
(ATAC-Seq) 

: 1

In [42]:
for i in {3,4,7,8}; 
do echo ${i};
   samtools sort output/aligned_files/SRR182215${i}_notMT_aligned.sam -o output/aligned_files/SRR182215${i}_notMT_aligned_sorted.bam;
   samtools index output/aligned_files/SRR182215${i}_notMT_aligned_sorted.bam;
   done

3[?2004h
4
7
8
(ATAC-Seq) 

: 1

In [43]:
ls output/aligned_files

SRR1822153_notMT_aligned.sam
SRR1822153_notMT_aligned_sorted.bam
SRR1822153_notMT_aligned_sorted.bam.bai
SRR1822154_notMT_aligned.sam
SRR1822154_notMT_aligned_sorted.bam
SRR1822154_notMT_aligned_sorted.bam.bai
SRR1822157_notMT_aligned.sam
SRR1822157_notMT_aligned_sorted.bam
SRR1822157_notMT_aligned_sorted.bam.bai
SRR1822158_notMT_aligned.sam
SRR1822158_notMT_aligned_sorted.bam
SRR1822158_notMT_aligned_sorted.bam.bai
(ATAC-Seq) 

: 1

## Quality Controls and Filters

The various outputs of this part, located in `output/stats/` must be studied to see whether something has gone wrong with the alignment or previous steps. What to do when you have an unexpected behavior will very much depend on your experiment.

### SAMtools flags
SAMtools flags give several information about a read, such as whether it is paired or not. Here we check the distribution of those flags within our sample.

The [wikipedia page](https://en.wikipedia.org/wiki/SAM_(file_format)) about SAM files does explicit the meaning of each SAMtools flag.

In [45]:
for i in {3,4,7,8}; 
do echo ${i};
   samtools flagstat output/aligned_files/SRR182215${i}_notMT_aligned_sorted.bam > output/stats/SRR182215${i}_aligned_samflags.txt;
   done

3[?2004h
4
7
8
(ATAC-Seq) 

: 1

### Samtools MAPQ score
The MAPQ score is related to the probability that a particular read is misaligned. The higher the MAPQ score is, the lower the risk that the read is misaligned is. In our experiment we used a particular threshold of 22, corresponding to a probability of misalignment of 10<sup>-12</sup>. 
Let's create the MAPQ score distribution using SAMtools for reads that have the flag that corresponds to 2 (each segment properly aligned according to aligner).

In [46]:
for i in {3,4,7,8}; 
do echo ${i};
   samtools view -f2 output/aligned_files/SRR182215${i}_notMT_aligned_sorted.bam | awk '{print $5}' | sort -n | uniq -c | sed 's/^ *//g'  > output/stats/SRR182215${i}_aligned_aligned_mapq.txt;
done

3[?2004h
4
7
8
(ATAC-Seq) 

: 1

Using Bowtie2, the reads that mapped several times have a MAPQ score of 1. The reads with MAPQ > 22 are uniquely mapped with good quality. We will now filter the reads to keep only these ones.   

### Filter base on the quality scores threshold you choose

In [48]:
for i in {3,4,7,8}; 
do echo ${i};
   samtools view -f2 -q22 -b output/aligned_files/SRR182215${i}_notMT_aligned_sorted.bam > output/aligned_files/SRR182215${i}_notMT_aligned_filtered.bam;
   done

3[?2004h
4
7
8
(ATAC-Seq) 

: 1

### Template length

Let's observe the Template Length. This is the distance between the mapped end of the DNA fragment and the mapped start of the DNA fragment, inclusively. If the mapping is correct, it corresponds to the size of the original DNA fragment. 

In [49]:
for i in {3,4,7,8}; 
do echo ${i};
   samtools view output/aligned_files/SRR182215${i}_notMT_aligned_filtered.bam | cut -f 9| sed 's/^-//' | sort -n | uniq -c |sed  's/^ *//g' > output/stats/SRR182215${i}_aligned_length.txt;
done

3[?2004h
4
7
8
(ATAC-Seq) 

: 1

### fastQC - general quality check

In [50]:
mkdir output/stats/fastQC/
fastqc -o output/stats/fastQC/ output/aligned_files/SRR182215{3,4,7,8}_notMT_aligned_filtered.bam

Started analysis of SRR1822153_notMT_aligned_filtered.bam
Approx 5% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 10% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 15% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 20% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 25% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 30% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 35% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 40% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 45% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 50% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 55% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 60% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 65% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 70% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 75% complete for SRR1822153_notMT_aligned_filtered.bam
Approx 80% co

: 1

Same as before, if you are treating several files at once, multiQC can be a good way to check whether the quality vary depending on each sample/condition, which might affect the results later on.

In [51]:
multiqc output/stats/fastQC/ -o output/stats/fastQC/


  /// ]8;id=546198;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.11

|           multiqc | Search path : /home/mag/ATACseq/TestJupyterNotebook/ATAC-seq/output/stats/fastQC
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 8/8    
|            fastqc | Found 4 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : output/stats/fastQC/multiqc_report.html
|           multiqc | Data        : output/stats/fastQC/multiqc_data
|           multiqc | MultiQC complete
(ATAC-Seq) 

: 1

## Peak calling

To call the ATAC-Seq peaks, we use MACS2. It might be more efficient to pool the reads from similar samples. You can do it by giving several bam files when calling MACS2. If you have several conditions that you want to compare, keep them separated and create one peak file per condition.

This part might need to be adapted to your particular case, so let's break down the various arguments : 
* `-t output/aligned_filtered/SRR182215{3,4,7,8}_notMT_aligned_filtered.bam` name of the file (or files) on which to do the peak calling. If you input several files, they will be pooled to give a single callpeak file.
* `-f BAM` Type of the output file desired
* `-g 1.2e7` Effective length of the genome (S.cerevisiae here). This has to be adapted to your genome (some are precompiled : hs: 2.7e9; mm: 1.87e9; ce: 9e7; dm: 1.2e8)
* `-q 0.05` minimum q-value for report of the peaks
* `-n output/ATAC_Seq_peaks/SRR182215{3,4,7,8}` base name of the output, several files containing a variety results will be generated


In [8]:
# I create one files with the peaks from all the samples. 
mkdir -p output/ATAC_Seq_peaks
macs2 callpeak -t output/aligned_files/SRR182215{3,4,7,8}_notMT_aligned_filtered.bam -f BAMPE -g 1.2e7 -q 0.05 -n output/ATAC_Seq_peaks/All_samples

INFO  @ Mon, 27 Sep 2021 14:52:08: 
# Command line: callpeak -t output/aligned_files/SRR1822153_notMT_aligned_filtered.bam output/aligned_files/SRR1822154_notMT_aligned_filtered.bam output/aligned_files/SRR1822157_notMT_aligned_filtered.bam output/aligned_files/SRR1822158_notMT_aligned_filtered.bam -f BAMPE -g 1.2e7 -q 0.05 -n output/ATAC_Seq_peaks/All_samples
# ARGUMENTS LIST:
# name = output/ATAC_Seq_peaks/All_samples
# format = BAMPE
# ChIP-seq file = ['output/aligned_files/SRR1822153_notMT_aligned_filtered.bam', 'output/aligned_files/SRR1822154_notMT_aligned_filtered.bam', 'output/aligned_files/SRR1822157_notMT_aligned_filtered.bam', 'output/aligned_files/SRR1822158_notMT_aligned_filtered.bam']
# control file = None
# effective genome size = 1.20e+07
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
#

: 1

### Mask regions

Some peaks might be in regions known to create artefactual peaks. It is important to remove the corresponding peaks from the results. 

The blacklists can be found [here](http://mitra.stanford.edu/kundaje/akundaje/release/blacklists/) or from [ENCODE](https://github.com/Boyle-Lab/Blacklist/tree/master/lists) and should be adapted to your project. In yeast, there is no such region and we only remove rDNA peaks.  

In [9]:
bedtools subtract -A -a output/ATAC_Seq_peaks/All_samples_summits.bed -b data/blacklist/sacCer3_rDNA.bed > output/ATAC_Seq_peaks/All_samples_summits_BL.bed

(ATAC-Seq) 

: 1

In [13]:
bedtools subtract -A -a output/ATAC_Seq_peaks/All_samples_peaks.narrowPeak -b data/blacklist/sacCer3_rDNA.bed > output/ATAC_Seq_peaks/All_samples_peaks_BL.bed

(ATAC-Seq) 

: 1

### Peak coverage per sample

We use bedtools coverage to get the peak coverage of every sample. 

In [14]:
mkdir output/peak_coverage/
for i in {3,4,7,8}; 
do echo ${i};
   bedtools coverage -a output/ATAC_Seq_peaks/All_samples_peaks_BL.bed -b output/aligned_files/SRR182215${i}_notMT_aligned_filtered.bam > output/peak_coverage/SRR182215${i}_peaks.bed;
   done

mkdir: impossible de créer le répertoire « output/peak_coverage/ »: Le fichier existe
3[?2004h(ATAC-Seq) 
4
7
8
(ATAC-Seq) 

: 1

Those outputs can be used to run differential analysis between 2 conditions with DEseq2 or edgeR. 

In [15]:
head output/peak_coverage/SRR1822153_peaks.bed

I	38	709	output/ATAC_Seq_peaks/All_samples_peak_1	489	.	12.4987	52.8872	48.9184	164	157	671	671	1.0000000
I	20734	21103	output/ATAC_Seq_peaks/All_samples_peak_2	92	.	5.21553	11.216	9.28764	192	20	369	369	1.0000000
I	29801	29980	output/ATAC_Seq_peaks/All_samples_peak_3	40	.	3.08132	5.65106	4.08756	62	21	179	179	1.0000000
I	32538	33302	output/ATAC_Seq_peaks/All_samples_peak_4	94	.	4.20471	11.3589	9.42615	444	38	634	764	0.8298429
I	34434	34710	output/ATAC_Seq_peaks/All_samples_peak_5	69	.	3.65353	8.70275	6.93077	77	30	276	276	1.0000000
I	45291	45875	output/ATAC_Seq_peaks/All_samples_peak_6	254	.	8.48398	28.3009	25.4309	139	38	555	584	0.9503425
I	58135	58783	output/ATAC_Seq_peaks/All_samples_peak_7	77	.	4.74139	9.55996	7.72537	396	25	619	648	0.9552469
I	61120	61495	output/ATAC_Seq_peaks/All_samples_peak_8	29	.	3.0819	4.48194	2.99934	282	11	368	375	0.9813333
I	68034	68916	output/ATAC_Seq_peaks/All_samples_peak_9	227	.	5.55434	25.4522	22.7533	508	119	882	882	1.0000000
I	70956	71510	output/AT

: 1

## Output description

From MACS https://hbctraining.github.io/Intro-to-ChIPseq/lessons/05_peak_calling_macs.html

From bedtools https://bedtools.readthedocs.io/en/latest/content/tools/coverage.html

    The number of features in B that overlapped (by at least one base pair) the A interval.
    The number of bases in A that had non-zero coverage from features in B.
    The length of the entry in A.
    The fraction of bases in A that had non-zero coverage from features in B.

Column description:
1. chr 
2. start 
3. end 
4. peak_ID 
5. MACS_score  
6. MACS_strand 
7. MACS_fold_enrichment 
8. MACS_-log10(pval) 
9. MACS_-log10(qval) 
10. MACS_summit (0-based offset from chrom start)
11. reads_in_peak 
12. peak_non-zero_bases 
13. peak_length 
14. peak_fraction_covered